In [ ]:
#importing the libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_score
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
import warnings
warnings.filterwarnings(action="ignore")

#importing dataset
file_paths = [
    "D:/AMINA/PFE24/datasets/commonvoice/cv-valid-train.csv",
    "D:/AMINA/PFE24/datasets/commonvoice/cv-other-train.csv",
    "D:/AMINA/PFE24/datasets/commonvoice/cv-invalid.csv"
]

dfs_train = []
for i in file_paths:
    df_train = pd.read_csv(i)
    dfs_train.append(df_train)

df_train = pd.concat(dfs_train, ignore_index=True)

file_paths2 = [
    "D:/AMINA/PFE24/datasets/commonvoice/cv-valid-dev.csv",
    "D:/AMINA/PFE24/datasets/commonvoice/cv-other-dev.csv",
]
dfs_val = []
for i in file_paths2:
    df_val = pd.read_csv(i)
    dfs_val.append(df_val)
df_val = pd.concat(dfs_val, ignore_index=True)

file_paths3 = [
    "D:/AMINA/PFE24/datasets/commonvoice/cv-valid-test.csv",
    "D:/AMINA/PFE24/datasets/commonvoice/cv-other-test.csv",
]
dfs_test = []
for i in file_paths3:
    df_test = pd.read_csv(i)
    dfs_test.append(df_test)

df_test = pd.concat(dfs_test, ignore_index=True)

#data cleaning
df_train = df_train.dropna(subset=['age'])
df_train = df_train[["filename","age","gender","accent"]]
df_train["gender"].fillna("other", inplace=True)
df_train = df_train.dropna(subset=['accent'])
#same for validation set
df_val = df_val.dropna(subset=['age'])
df_val = df_val[["filename","age","gender","accent"]]
df_val["gender"].fillna("other", inplace=True)
df_val = df_val.dropna(subset=['accent'])
#same for testing set
df_test = df_test.dropna(subset=['age'])
df_test = df_test[["filename","age","gender","accent"]]
# Convert 'age' column to numerical
cleanup_nums = {"age": {"teens":1.0,"twenties":2.0,"thirties":3.0,"fourties":4.0,"fifties":5.0,"sixties":6.0,"seventies":7.0,"eighties":8.0}}
df_test = df_test.replace(cleanup_nums)
df_train = df_train.replace(cleanup_nums)
#Convert categorical variables to numerical using one-hot encoding
df_train = pd.get_dummies(df_train, columns=['gender', 'accent'])
df_test = pd.get_dummies(df_test, columns=['gender', 'accent'])


In [ ]:
# Define the model
model = Sequential([#define  a linear stack of layers for the mode
    Embedding(input_dim=len(df_train.columns)-1, output_dim=64, input_length=1),#map categorical variables to dense vectors.
    LSTM(units=64, activation='tanh', dropout=0.2, recurrent_dropout=0.2),#work for sequential data
    Dense(32, activation='relu'),#fully connected layer with 32 neurons and ReLU activation function
    Dropout(0.5),#prevent overfitting by randomaly setting fraction of input units to 0 at each update
    Dense(8, activation='softmax')  # Softmax activation for multi-class classification to output probabilities for each class
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# Adam is an optimization algorithm that is an extension to stochastic gradient descent
#Sparse categorical cross-entropy is commonly used in multi-class classification problems where the labels are integers

# Define training and validation data
X_train = df_train.drop(columns=['filename', 'age']).values
y_train = df_train['age'].values - 1  # Convert labels to start from 0
X_val = df_val.drop(columns=['filename', 'age']).values
y_val = df_val['age'].values - 1  # Convert labels to start from 0

# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))
# An epoch is one complete pass through the entire training dataset.

# Evaluate the model on test data
X_test = df_test.drop(columns=['filename', 'age']).values
y_test = df_test['age'].values - 1  # Convert labels to start from 0
loss, accuracy = model.evaluate(X_test, y_test)

print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')

In [ ]:
# Define preprocessing steps
filename_preprocessor = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),  
    ('encoder', OneHotEncoder())  
])

text_preprocessor = Pipeline([
    ('tfidf', TfidfVectorizer()),  
])

# Combine preprocessing steps for all features
preprocessor = ColumnTransformer([
    ('filename', filename_preprocessor, ['filename']),
    ('text', text_preprocessor, 'text'),  
    ('other', 'passthrough', df.select_dtypes(exclude="object").columns)  
])

# Split data into X (predictors) and y (target)
X = df.drop(columns=['age'])
y = df['age']

# Split text data
text_data = X['text']

# Tokenize text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_data)
X_text = tokenizer.texts_to_sequences(text_data)
X_text = pad_sequences(X_text, maxlen=100)

# Split the data
X_train, X_test, X_text_train, X_text_test, y_train, y_test = train_test_split(X, X_text, y, test_size=0.2, random_state=42)

# Define input layers for numeric and text data
numeric_input = Input(shape=(X_train.shape[1],))
text_input = Input(shape=(X_text_train.shape[1],))  # Use the shape of preprocessed text data

# Define embedding layer for categorical data (filename)
filename_embedding = Embedding(input_dim=len(df['filename'].unique()), output_dim=64, input_length=1)(numeric_input)

# LSTM layer for text data
lstm_layer = LSTM(units=64, activation='tanh', dropout=0.2, recurrent_dropout=0.2)(text_input)

# Concatenate numerical and text embeddings
concatenated = concatenate([filename_embedding, lstm_layer])

# Additional layers
dense_layer = Dense(32, activation='relu')(concatenated)
dropout_layer = Dropout(0.5)(dense_layer)

# Output layer
output_layer = Dense(8, activation='softmax')(dropout_layer)

# Define the model
model = Model(inputs=[numeric_input, text_input], outputs=output_layer)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit([X_train, X_text_train], y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model
loss, accuracy = model.evaluate([X_test, X_text_test], y_test)
print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')